In [2]:
# https://www.kaggle.com/competitions/ban-674-final

# The goal is to predict the quarterly sales to each of the 75 customers


In [3]:
# Importing necessary Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error
import warnings
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
warnings.filterwarnings('ignore')

In [4]:
# Get input data
train = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
econ_df = pd.read_csv("EconomicIndicators.csv")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 675 non-null    int64  
 1   Company            675 non-null    object 
 2   Quarter            675 non-null    object 
 3   QuickRatio         675 non-null    float64
 4   InventoryRatio     523 non-null    float64
 5   RevenueGrowth      675 non-null    float64
 6   MarketshareChange  675 non-null    float64
 7   Bond rating        675 non-null    object 
 8   Stock rating       675 non-null    object 
 9   Region             675 non-null    object 
 10  Industry           675 non-null    object 
 11  Sales              525 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 63.4+ KB


In [6]:
# Adding new feature "Quarter_Category" which categorises Q1, Q5, Q9 to '1', Q2, Q6 to '2', Q3, Q7 to '3', Q4, Q8 to '0'
data = {'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9']}

def new_feature(sample_df):
    df_new = pd.DataFrame(sample_df)
    df_new['Quarter_Num'] = df_new['Quarter'].str.extract('(\d+)').astype(int)
    df_new['Quarter_Category'] = (df_new['Quarter_Num']) % 4
    return df_new

train_new = new_feature(train)
test_new = new_feature(test_df)

In [7]:
train_new.head()

,ID,Company,Quarter,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange,Bond rating,Stock rating,Region,Industry,Sales,Quarter_Num,Quarter_Category
0,0,CMP01,Q1,2.02,7.71,0.05,-0.04,CCC,Buy,South,Metal Fabrication,1517.0,1,1
1,1,CMP01,Q2,2.01,4.10,0.03,0.00,CCC,Hold,South,Metal Fabrication,2968.0,2,2
2,2,CMP01,Q3,2.02,6.79,0.06,-0.02,CCC,Buy,South,Metal Fabrication,1497.0,3,3
3,3,CMP01,Q4,1.98,3.97,0.01,0.02,CCC,Buy,South,Metal Fabrication,2929.0,4,0
4,4,CMP01,Q5,1.96,7.41,-0.07,0.02,CCC,Buy,South,Metal Fabrication,1452.0,5,1


In [8]:
test_new.head()

,ID,Company,Quarter,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange,Bond rating,Stock rating,Region,Industry,Quarter_Num,Quarter_Category
0,7,CMP01,Q8,1.93,2.79,-0.03,-0.01,CCC,Buy,South,Metal Fabrication,8,0
1,8,CMP01,Q9,1.93,4.77,0.00,0.00,CCC,Buy,South,Metal Fabrication,9,1
2,16,CMP02,Q8,1.97,2.34,0.04,-0.03,A,Sell,West,Infrastructure,8,0
3,17,CMP02,Q9,1.93,4.14,0.04,0.01,BBB,Hold,West,Infrastructure,9,1
4,25,CMP03,Q8,0.67,NaN,-0.05,-0.01,BB,Buy,East,Infrastructure,8,0


In [9]:
# Removing the duplicate rows
train_new = train_new.sort_values(by='Sales', ascending=False)  # Sort by 'sales' to bring non-null values first
train_new = train_new.drop_duplicates(subset=['Company', 'Quarter'], keep='first')

In [10]:
# Create a mapping for quarters to months
quarter_to_month = {
    'Q1': [1, 2, 3],
    'Q2': [4, 5, 6],
    'Q3': [7, 8, 9],
    'Q4': [10, 11, 12],
    'Q5': [13, 14, 15],
    'Q6': [16, 17, 18],
    'Q7': [19, 20, 21],
    'Q8': [22, 23, 24],
    'Q9': [25, 26, 27, 28] 
}
# Function to calculate average economic indicators for each quarter
def average_econ_indicators(econ_df, quarter_to_month):
    # Initialize a list to store averaged economic indicators
    averaged_indicators = []

    for quarter, months in quarter_to_month.items():
        # Filter economic indicators for the months in the current quarter
        econ_data_for_quarter = econ_df[econ_df['Month'].isin(months)]
        # Drop the Month column
        econ_data_for_quarter = econ_data_for_quarter.drop(columns=['Month'])
        # Calculate the average for each indicator
        average_econ_data = econ_data_for_quarter.mean(numeric_only=True)
        # Assign the quarter to the averaged data
        average_econ_data['Quarter'] = quarter
        # Append the averaged data to the result list
        averaged_indicators.append(average_econ_data)

    # Convert the list of averaged data to a DataFrame
    averaged_indicators_df = pd.DataFrame(averaged_indicators)
    return averaged_indicators_df

# Calculate average economic indicators
average_econ_df = average_econ_indicators(econ_df, quarter_to_month)

# Merge the train/test datasets with the averaged economic indicators
def merge_with_econ_data(df, average_econ_df):
    return pd.merge(df, average_econ_df, on='Quarter')

# Merge train and test datasets with the averaged economic indicators
train_with_econ = merge_with_econ_data(train_new, average_econ_df)
test_with_econ = merge_with_econ_data(test_new, average_econ_df)

In [11]:
# The train dataset has Q8, Q9 quarters which is present in test data set, hence removing it.
train_with_econ = train_with_econ[~train_with_econ['Quarter'].isin(['Q8', 'Q9'])]

In [12]:
# Impute missing Sales values using regression within each company

from sklearn.linear_model import LinearRegression
def impute_missing_sales(df):
    companies = df['Company'].unique()
    for company in companies:
        company_data = df[df['Company'] == company].sort_values(by='Quarter')
        missing_sales = company_data['Sales'].isna()
        if missing_sales.sum() > 0:
            available_data = company_data[~missing_sales]
            missing_data = company_data[missing_sales]
            if not available_data.empty:
                # Use linear regression to predict missing values
                lr = LinearRegression()
                quarters = np.array(range(len(available_data))).reshape(-1, 1)
                sales = available_data['Sales'].values
                lr.fit(quarters, sales)
                missing_quarters = np.array(range(len(available_data), len(available_data) + len(missing_data))).reshape(-1, 1)
                df.loc[missing_data.index, 'Sales'] = lr.predict(missing_quarters)
            else:
                # If no available data, use mean of previous and next quarter
                for idx in missing_data.index:
                    prev_idx = df.index[df.index < idx][-1]
                    next_idx = df.index[df.index > idx][0]
                    prev_sales = df.loc[prev_idx, 'Sales'] if prev_idx else np.nan
                    next_sales = df.loc[next_idx, 'Sales'] if next_idx else np.nan
                    if not np.isnan(prev_sales) and not np.isnan(next_sales):
                        df.loc[idx, 'Sales'] = (prev_sales + next_sales) / 2
    return df

# Impute missing Sales in the training data
train_with_econ = impute_missing_sales(train_with_econ)

In [13]:
# Define features and target
X = train_with_econ.drop(columns=['Quarter','ID','Sales'])
y = train_with_econ['Sales']
X_test = test_with_econ.drop(columns=['Quarter', 'ID'])

# Preprocessing pipelines for both numeric and categorical features
numeric_features = ['QuickRatio', 'InventoryRatio', 'RevenueGrowth', 'MarketshareChange',
                    'Consumer Sentiment', 'Interest Rate','Quarter_Category',
                    'PMI', 'Money Supply', 'NationalEAI', 'EastEAI', 'WestEAI', 'SouthEAI', 'NorthEAI']
categorical_features = ['Company', 'Region', 'Industry']

# Define custom label encoder for ordinal features
ordinal_features = ['Bond rating', 'Stock rating']
bond_rating_order = ['CCC', 'B', 'BB', 'BBB', 'A', 'AA', 'AAA']
stock_rating_order = ['Strong Sell', 'Sell', 'Hold', 'Buy', 'Strong Buy']

# Define custom label encoder for ordinal features
ordinal_features = ['Bond rating', 'Stock rating']

# Create pipelines for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[bond_rating_order, stock_rating_order]))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [15]:
# Define the model
from sklearn.tree import DecisionTreeRegressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
 ('regressor', RandomForestRegressor(n_estimators=100, random_state=40))
    # ('regressor', LGBMRegressor(learning_rate=0.01, random_state=42))
    # ('XGBoost', XGBRegressor(max_depth=5, n_estimators=100, random_state=42))
])


In [16]:
# Since this is time series data cross-validation is performed using TimeSeriesSplit function

tscv = TimeSeriesSplit(n_splits=5)

# Cross-validate the model using TimeSeriesSplit
cv_results = cross_val_score(model, X, y, cv=tscv, scoring='neg_mean_absolute_error')
print(f'Cross-validated MAE: {-cv_results.mean()}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000176 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 107
[LightGBM] [Info] Number of data points in the train set: 90, number of used features: 12
[LightGBM] [Info] Start training from score 4245.400000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [17]:
from sklearn.model_selection import cross_val_score, KFold

# Define KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validate the model using KFold
cv_results = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error')

# Print the mean cross-validated MAE
print(f'Cross-validated MAE: {-cv_results.mean()}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000188 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 337
[LightGBM] [Info] Number of data points in the train set: 420, number of used features: 23
[LightGBM] [Info] Start training from score 3449.707143
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [19]:
# Fit the model on the entire training set and predict on the test set
model.fit(X, y)
y_test_pred = model.predict(X_test)

In [21]:
submission = pd.DataFrame({
    'ID': test_with_econ['ID'],
    'Sales': y_test_pred
})

# Save to CSV
submission.to_csv('submission.csv', index=False)